In [6]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_binary
from io import BytesIO
import base64
from python3_anticaptcha import ImageToTextTask, CallbackClient
from PIL import Image
from time import sleep
import requests
import time
import config as cg

idCard = cg.IDCardInfo()
anticaptcha = cg.AntiCaptchaInfo()


# 前臺開啟瀏覽器模式
def openChrome():
    # 加啟動配置
    option = webdriver.ChromeOptions()
    option.add_argument('disable-infobars')
    option.add_argument("--window-size=1280,1024")
    # 開啟chrome瀏覽器
    driver = webdriver.Chrome(chrome_options=option)
    return driver

# 輸入查詢表單
def insertIDCardReissue(driver):
    url = "https://www.ris.gov.tw/app/portal/3043"
    driver.get(url)
    driver.implicitly_wait(10)
    driver.switch_to.frame("content-frame")
    
    # 找到身分證字號輸入框並輸入查詢內容
    driver.find_element_by_id("idnum").send_keys(idCard.id)
    
    # 找到發證日期 下拉選單
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyTWY')).select_by_value(idCard.applyTWY)
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyMM')).select_by_value(idCard.applyMM)
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyDD')).select_by_value(idCard.applyDD)
        
    #發證地點：
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('siteId')).select_by_value(idCard.siteId)

    #領補換類別：
    driver.implicitly_wait(1)
    Select(driver.find_element_by_id('applyReason')).select_by_value(idCard.applyReason)

    #圖片驗證 示範錯誤
    driver.implicitly_wait(1)
    captchaInput_captcha = driver.find_element_by_id("captchaInput_captcha-refresh")
    path = 'captchaImage.png'
    
    captchaInput_captcha.send_keys("AAAAA")
    
    # 提交表單
    
    result = None
    while result is None:
        driver.find_element_by_xpath("/html/body/div/div[4]/div[1]/div/form/div[4]/button[1]").click()
        
        try:
            result = send_form(driver, path)
        except:
            pass
    
    print(result)        

def send_form(driver, path):
    
    try:
        driver.find_element_by_id("resultBlock")
    
    except:
        driver.set_window_size(1280, 1024)
        captchaInput = driver.find_element_by_id("captchaInput_captcha-refresh")
        captchaInput.clear()
        get_captcha(driver, path)
        captcha_key = parse_captcha(path)
        print("send_form.parse_captcha.captcha_key = ", captcha_key)
        
        captchaInput.send_keys(captcha_key)
    else:
        return '表單送出成功'
    
def get_captcha(driver, path):
    scroll_width = driver.execute_script('return document.body.parentNode.scrollWidth')
    scroll_height = driver.execute_script('return document.body.parentNode.scrollHeight')
    driver.set_window_size(scroll_width, scroll_height) #the trick
    time.sleep(1)
    png = driver.get_screenshot_as_png()         # 先將目前的 screen 存起來
    image = Image.open(BytesIO(png))
    image = image.crop((350, 200, 580, 290)) # 根據位置剪裁圖片
    image.save(path)               # 重新儲存圖片為 png 格式
    
def parse_captcha(image_path):
    # Load image into memory
    buffer = BytesIO()
    image = Image.open(image_path)
    image.save(buffer, format="png")
    # Use base64 to encode image buffer
    img_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    # Anti-captcha API structure
    data = {
        "clientKey":anticaptcha.key,
        "task": {
            "type": "ImageToTextTask",
            "body": img_str,
            "phrase":False,
            "case": True,
            "numeric": 0,
            "math": 0,
            "minLength": 5,
            "maxLength": 5
        }
    }
    # Create a ImageToTextTask and retrieve taskId from response
    r = requests.post("https://api.anti-captcha.com/createTask", json=data)
    r.raise_for_status()
    task_id = r.json()['taskId']
    # Polling for task finish.
    ret = ""
    while True:
        data = {
            "clientKey":anticaptcha.key,
            'taskId': task_id
        }
        r = requests.post("https://api.anti-captcha.com/getTaskResult", json=data)
        r.raise_for_status()
        if r.json()['status'] == 'ready':
            ret = r.json()['solution']['text']
            break
        time.sleep(3)
    return ret

In [7]:
driver=openChrome()
insertIDCardReissue(driver)

<ipython-input-6-cb0aedfc44dd>:24: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=option)


send_form.parse_captcha.captcha_key =  S7FHU
表單送出成功
